# Observations for the night June 11th 2021 


https://harvard.zoom.us/rec/play/N7umVF8Rk3vWvW_O3jkaTinofB2WxW3QKz0fVai_K-qvAYLt37uMrvskOlnBiMEjduSeQVzWT7_DDYzc.IY7Pqjef82YlWmZE?continueMode=true&_x_zm_rtaid=BjQYrDaETYeYRUMw8NQgLw.1613559822570.c813d210d38e378b719cb4048493df5b&_x_zm_rhtaid=205


- work with Weakly_2021_21
- use jupyter kernel LSST
- requires the installation see https://github.com/stubbslab/PCWG-AuxTel/blob/main/setup_script.md
- requires to write in file {HOME}/notebooks/.user_setups


In [1]:
# ! eups list -s

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [3]:
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend("lsst.display.firefly")

In [4]:
# correct for bug in stack in wl_2021_02
%config IPCompleter.use_jedi=False

In [5]:
FLAG_SAVE_IMAGE=True
FLAG_SAVE_IMAGE=True

In [6]:
!ls /project/shared/auxTel/rerun/mfl/slurmRun

calexp	icExp  postISRCCD	     repositoryCfg.yaml  spectractorOutput
config	icSrc  processStar_metadata  schema


In [7]:
!ls /project/shared/auxTel

CALIB	   CALIB-OLD	 CALIB-RHL  _parent	 ref_cats
CALIB_OLD  calibrations  Fossil     _parent.old  rerun


# Configuration

## Location and date

In [8]:
from lsst.rapid.analysis.nightReport import NightReporter
# data exist here
repoDir='/project/shared/auxTel'
# recommended but wrong
#repoDir='project/shared/auxTel/rerun/mfl/slurmRun'
dayObs="2021-06-11"
reporter=NightReporter(repoDir,dayObs)

/home/dagoret/repos/rapid_analysis/python/lsst/rapid/analysis/nightReport.py:79: FutureWarning: Gen2 Butler has been deprecated (Butler). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  self._butler = dafPersist.Butler(repoDir)
/home/dagoret/repos/rapid_analysis/python/lsst/rapid/analysis/nightReport.py:79: FutureWarning: Gen2 Butler has been deprecated (LatissMapper). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  self._butler = dafPersist.Butler(repoDir)


IndexError: list index out of range

# Selection

In [ ]:
DISPERSER_SEL="holo4_003"

# Information from reporter

In [ ]:
print(reporter.getObservedObjects())

In [ ]:
reporter.makePolarPlotForObjects()

In [ ]:
reporter.plotPerObjectAirMass()

In [ ]:
reporter.printObsTable()

In [ ]:
alert False

# Butler

## Configuration for Butler

In [ ]:
#butler=dafPersist.Butler("/project/shared/auxTel")
butler=dafPersist.Butler(repoDir)

## Sequence selection

In [ ]:
all_myseq_holo=range(216,276)
all_myseq=all_myseq_holo

In [ ]:
seqNum=all_myseq_holo[0]

In [ ]:
seqNum

## Display

In [ ]:
display1=afwDisplay.getDisplay(frame=1,port=80,name='sylviedc_debug_2021_06_10',url=os.environ["FIREFLY_URL"])

In [ ]:
raw=butler.get('raw',dayObs=dayObs,seqNum=seqNum)

In [ ]:
display1.mtv(raw)

In [ ]:
display1.getClient().display_url()

In [ ]:
print(butler.queryMetadata('raw','seqNum',dayObs=dayObs))

In [ ]:
#exp=butler.get('quickLookExp',dayObs=dayObs,seqNum=seqNum)

## Meta Data

In [ ]:
md=butler.get("raw_md",dayObs=dayObs,seqNum=seqNum)

In [ ]:
md.toDict()

# Start

In [ ]:
plt.rcParams["axes.labelsize"] = 'large'
plt.rcParams['axes.titlesize'] = 'large'
plt.rcParams['xtick.labelsize']= 'large'
plt.rcParams['ytick.labelsize']= 'large'

In [ ]:
all_my_raw_array=[]
all_my_header=[]
all_my_object=[]
all_my_seq=[]
all_my_grating=[]
all_my_filter=[]
all_my_date=[]


for seqNum in all_myseq:
    raw=butler.get('raw',dayObs=dayObs,seqNum=seqNum)
    md=butler.get("raw_md",dayObs=dayObs,seqNum=seqNum)
    
    header=md.toDict()
    
    #decode header
    
    obj_name=header['OBJECT']
    obj_name2 = ''.join(obj_name.split())
    grating=header['GRATING']
    obsid=header['OBSID']
    date=obsid.replace('AT_O_','') # remove pre-tag to get date only and seqnum
    filter_name=header['FILTER']
    filter_name2=filter_name.split("_")[0]   # cut empty_1 to empty
    
    # select only pre-selected disperser
    if grating == DISPERSER_SEL: 
        all_my_raw_array.append(np.copy(raw.image.array))
        all_my_header.append(header)
        all_my_object.append(obj_name2)
        all_my_seq.append(header['SEQNUM'])
        all_my_grating.append(grating)
        all_my_filter.append(filter_name2)
        all_my_date.append(date)

## Total number of images

In [ ]:
NSEL=len(all_my_raw_array)

In [ ]:
NSEL

# Plot

In [ ]:
idx=0

In [ ]:
fig = plt.figure(figsize=(10,10))
ax=fig.add_subplot(111)
ax.imshow(all_my_raw_array[idx],origin='lower',cmap="jet",norm=LogNorm())
plt.show()

In [ ]:

Ncol=4
Nrow=2
Ntot=Ncol*Nrow

fig = plt.figure(figsize=(20,20))

# loop on angle
for idx in np.arange(Ntot):
    if idx>=NSEL:
        break
        
    ax=fig.add_subplot(Nrow,Ncol,idx+1)
    ax.imshow(all_my_raw_array[idx],origin='lower',cmap="jet",norm=LogNorm())
    ax.set_title(all_myseq[idx]) 
    ax.minorticks_on()
    ax.set_aspect("equal")
    
    #ax2=ax.twiny()
    #ax2.imshow(all_my_raw_array[idx],origin='lower',cmap="jet",norm=LogNorm())
    #ax2.set_xlim(ax.get_xlim())
    #ax2.minorticks_on()
    
    #ax.tick_params(which='major', length=10, width=2, direction='inout')
    #ax.tick_params(which='minor', length=5, width=2, direction='inout')
    #ax.grid(which='both')



    #ax.set_aspect('equal')

    #x1, x2 = ax.get_xlim() 
    #y1, y2 = ax.get_ylim() 
plt.show()

In [ ]:
#all_my_raw_array=[]
#all_my_header=[]
#all_my_object=[]
#all_my_seq=[]
#all_my_grating=[]
#all_my_filter=[]


In [ ]:
if FLAG_SAVE_IMAGE:
  
    for idx in np.arange(NSEL):
        filename=all_my_grating[idx]+"_"+all_my_filter[idx]+"_"+all_my_object[idx]+"_"+all_my_date[idx]+".fits"
        print(filename)
    
        hdr = fits.Header()
    
        for key,value in all_my_header[idx].items():
            hdr[str(key)]=value
        
        hdu = fits.PrimaryHDU(data=all_my_raw_array[idx],header=hdr)  # with headers
        #hdu = fits.PrimaryHDU(data=all_my_raw_array[idx])
    
        hdul = fits.HDUList([hdu])
    
        hdul.writeto(filename,overwrite=True)
    
        